# Get started with Metrics Tracking

This notebook demonstrates how to use MLFlow to:
- Log metrics, params, and artifacts to MLFlow.
- Log, register, and load models using a local MLflow Tracking Server.
- Interact with the MLflow Tracking Server using the MLflow fluent API.
- Perform inference on Pandas DataFrames by loading models as generic Python Functions (pyfunc).

In [1]:
%load_ext autoreload
%autoreload 2

import joblib
import mlflow
from mlflow.models import infer_signature
import mlflow.sklearn
import pandas as pd
from pathlib import Path
from sklearn import ensemble, model_selection
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Train model and calculate metrics

## Load Data

More information about the dataset can be found in UCI machine learning repository: https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset

Acknowledgement: Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg

In [2]:
# Download original dataset with: python src/load_data.py 

raw_data = pd.read_csv("../data/raw_data.csv")
raw_data.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


## Prepare data

In [3]:
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'mnth', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday', ]

In [4]:
sample_data = raw_data.set_index('dteday').loc['2011-01-01 00:00:00':'2011-01-28 23:00:00'].reset_index()

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    sample_data[numerical_features + categorical_features],
    sample_data[target],
    test_size=0.3
)

print(X_train.shape)
print(X_test.shape)

(415, 10)
(179, 10)


## Train a  Model

In [5]:
model = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)
model.fit(X_train, y_train) 

model_path = Path('../models/model.joblib')
joblib.dump(model, model_path)

['../models/model.joblib']

In [6]:
model

,n_estimators,50
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


## Calculate Metrics

In [7]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

preds = model.predict(X_test)

me = mean_squared_error(y_test, preds)
mae = mean_absolute_error(y_test, preds)

print(me, mae)

334.5238972067039 11.789050279329608


# Metrics Tracking with MLflow

## Set up MLFlow

In [8]:
MLFLOW_TRACKING_URI = "http://0.0.0.0:5001"

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

mlflow.set_experiment("1-get-started")


2025/06/17 09:31:27 INFO mlflow.tracking.fluent: Experiment with name '1-get-started' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/679172873092264664', creation_time=1750127487556, experiment_id='679172873092264664', last_update_time=1750127487556, lifecycle_stage='active', name='1-get-started', tags={}>

In [9]:
# List experiments in MLflow
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/679172873092264664', creation_time=1750127487556, experiment_id='679172873092264664', last_update_time=1750127487556, lifecycle_stage='active', name='1-get-started', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/660324135451925515', creation_time=1750127114703, experiment_id='660324135451925515', last_update_time=1750127114703, lifecycle_stage='active', name='6-question-answering-evaluation', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/238283046149206350', creation_time=1750127039368, experiment_id='238283046149206350', last_update_time=1750127039368, lifecycle_stage='active', name='5-genai-with-mlflow-3', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1750123536195, experiment_id='0', last_update_time=1750123536195, lifecycle_stage='active', name='Default', tags={}>]

## Log params, metrics and artifacts

In [ ]:
with mlflow.start_run() as run: 

    # Log params 
    mlflow.log_param('model', 'RandomForest') 
    mlflow.log_params({'random_state': 0, 'n_estimators': 50})

    # Log metrics
    mlflow.log_metric('me', round(me, 3))
    mlflow.log_metric('mae', round(mae, 3))

    # Log artifacts (data and model files)
    mlflow.log_artifact("../data/raw_data.csv")
    mlflow.log_artifact("../models/model.joblib")

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("random-forest", "Random Forest Classifier")

    # Infer the model signature
    signature = infer_signature(X_train, model.predict(X_train))


    # Log the model, which inherits the parameters and metric
    model_info = mlflow.sklearn.log_model(
        sk_model=model,
        name="bikes_sharing_model_rf",
        signature=signature,
        input_example=X_train,
        registered_model_name="1-get-started",
    )


2025/06/17 09:32:09 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2025/06/17 09:32:09 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
/Users/mikhailrozhkov/dev/mlrepa/mlflow-1-metrics-tracking/.venv/lib/python3.11/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-

🏃 View run skittish-bass-903 at: http://0.0.0.0:5001/#/experiments/679172873092264664/runs/a313ad423c1a4050a048f96cdca765dd
🧪 View experiment at: http://0.0.0.0:5001/#/experiments/679172873092264664


## Load our saved model as a Python Function

Although we can load our model back as a native scikit-learn format with `mlflow.sklearn.load_model()`, below we are loading the model as a generic Python Function, which is how this model would be loaded for online model serving. We can still use the `pyfunc` representation for batch use cases, though, as is shown below.

In [13]:
model_info.model_uri

'models:/m-9d4e6b2494b44cb48e33fdd1bed7c1a6'

In [14]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

loaded_model

/Users/mikhailrozhkov/dev/mlrepa/mlflow-1-metrics-tracking/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


mlflow.pyfunc.loaded_model:
  artifact_path: mlflow-artifacts:/679172873092264664/models/m-9d4e6b2494b44cb48e33fdd1bed7c1a6/artifacts
  flavor: mlflow.sklearn
  run_id: a313ad423c1a4050a048f96cdca765dd

## Use `loaded_model` to get predictions for `X_test` dataset

In [15]:
predictions = loaded_model.predict(X_test)

# Convert X_test validation feature data to a Pandas DataFrame
result = pd.DataFrame(X_test)

# Add the actual classes to the DataFrame
result["actual_class"] = y_test

# Add the model predictions to the DataFrame
result["predicted_class"] = predictions

result[:4]

,temp,atemp,hum,windspeed,mnth,hr,weekday,season,holiday,workingday,actual_class,predicted_class
449,0.12,0.1212,0.30,0.2537,1,18,5,1,0,1,120,146.48
124,0.20,0.1970,0.40,0.2239,1,11,5,1,0,1,67,57.44
54,0.16,0.1364,0.69,0.3284,1,10,2,1,0,1,42,41.86
526,0.16,0.1515,0.69,0.2239,1,2,2,1,0,1,2,3.36
